<a href="https://colab.research.google.com/github/gcorneliusvh/semantic_chatbot/blob/master/Looker_%26_Gemini_Semantic_Chatbot_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Semantic Layer Chatbot: From Looker to LLM Agents

This notebook walks through the core logic of the "Looker + Gemini" chatbot. We will strip away all the Streamlit frontend code (`app.py`, `pages/`) and focus purely on the backend logic.

Our goal is to build a system that can:
1.  **Connect to Looker** using its Python SDK.
2.  **Understand the Looker data model (Explore)** by fetching its metadata.
3.  **Build a LangChain "Tool"** that allows an LLM to query Looker.
4.  **Create an "Agent"** that can use this tool to answer questions.
5.  **Build a "Router"** to create a multi-agent system that can hand off tasks between a **Looker Agent** and a **Python Data-Analysis Agent**.

---

## **Section 1: Setup and Credentials**

First, we need to install the required libraries and set up our API keys.

This system requires two sets of credentials:
1.  **Google Gemini API Key**: For powering the LLMs.
2.  **Looker SDK Credentials**: To allow the Python script to authenticate with your Looker instance.

In [ ]:
# This file is a Python script formatted as a teaching notebook.
# You can run it "cell" by "cell" in an IDE like VS Code (which supports
# # %% cell dividers) or a standard Jupyter Notebook environment.

# %%
!pip install "langchain>=0.2.0" langchain-google-genai looker-sdk pandas langchain-community langchain-experimental duckduckgo-search

In [ ]:
import os
import json
import pandas as pd
import io
from urllib.parse import urlencode

# --- LangChain Core ---
from langchain_core.tools import StructuredTool, tool
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain import hub
from langchain_core.messages import HumanMessage, AIMessage
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_core.tools import render_text_description

# --- LLM Imports ---
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Pydantic Schemas ---
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Literal

# --- Looker SDK ---
import looker_sdk
from looker_sdk import models40

# --- Other Tools ---
from langchain_community.tools import DuckDuckGoSearchRun

### **1.1. Set Up Your Credentials**

We use `os.environ` to load credentials. In a real notebook, you might use `userdata.get()` (in Colab) or another secrets manager.

**IMPORTANT:** Replace the placeholder values below with your actual credentials.

In [46]:
# --- 1. SET UP YOUR CREDENTIALS ---
# We use os.environ to load credentials.
# In a real notebook, you might use `userdata.get()` (in Colab) or another secrets manager.
#
# !!!!!!!!!!!!!!!!!!
# !! IMPORTANT:     !!
# !! REPLACE THESE  !!
# !! PLACEHOLDERS   !!
# !!!!!!!!!!!!!!!!!!
from google.colab import userdata
# 1a. Google Gemini API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# 1b. Looker SDK Credentials
# These are read by the looker_sdk.init40() function
os.environ["LOOKERSDK_BASE_URL"] = "https://igmprinting.cloud.looker.com:19999" # Must include :19999
os.environ["LOOKERSDK_CLIENT_ID"] = userdata.get('LOOKERSDK_CLIENT_ID')
os.environ["LOOKERSDK_CLIENT_SECRET"] = userdata.get('LOOKERSDK_CLIENT_SECRET')
os.environ["LOOKERSDK_VERIFY_SSL"] = "true"
os.environ["LOOKERSDK_TIMEOUT"] = "120"

# --- 2. SET UP BROWSER URL (for Embeds) ---
# This is the URL you use to access Looker in your browser (no :19999)
LOOKER_BROWSER_URL = "https://igmprinting.cloud.looker.com"

# --- 3. DEFINE MODEL AND EXPLORE ---
# These must match the Looker Explore you want to query
# We'll use the ones from the uploaded files
MODEL_NAME = "data_block_acs_bigquery"
EXPLORE_NAME = "acs_census_data"

---

## **Section 2: The Foundation - The Looker Semantic Layer**

An LLM cannot guess your database schema or business logic. To make it "semantically aware," we must provide it with **metadata**. This metadata, generated from your Looker model, is what "ground" the LLM, enabling it to map natural language ("total population") to the correct LookML field (`blockgroup.total_pop`).

### **2.1. Fetching Looker Metadata (One-Time-Setup)**

The first step is to run a script (like `01_fetch_metadata.py`) to connect to your Looker instance and pull the schema for your desired Explore. This generates a JSON file that the agent will use as its "brain."

In [47]:
def fetch_explore_metadata(sdk: looker_sdk.sdk.api40.methods.Looker40SDK, model_name: str, explore_name: str, output_filename: str):
    """
    Fetches all dimensions and measures for a given Looker Explore
    and saves them to a JSON file.

    This is based on 01_fetch_metadata.py
    """
    print(f"Fetching metadata for {model_name}::{explore_name}...")
    try:
        # 1. Get the explore definition from the Looker SDK
        explore = sdk.lookml_model_explore(
            lookml_model_name=model_name,
            explore_name=explore_name,
            fields="fields" # Request the 'fields' definition
        )

        fields_data = []

        # 2. Parse dimensions
        if explore.fields and explore.fields.dimensions:
            for dim in explore.fields.dimensions:
                fields_data.append({
                    "name": dim.name,
                    "label": dim.label_short or dim.label,
                    "description": dim.description,
                    "type": "dimension"
                })

        # 3. Parse measures
        if explore.fields and explore.fields.measures:
            for mea in explore.fields.measures:
                fields_data.append({
                    "name": mea.name,
                    "label": mea.label_short or mea.label,
                    "description": mea.description,
                    "type": "measure"
                })

        # 4. Save to a file
        with open(output_filename, "w") as f:
            json.dump(fields_data, f, indent=2)

        print(f"Successfully fetched {len(fields_data)} fields.")
        print(f"Metadata saved to {output_filename}")

    except Exception as e:
        print(f"Error fetching explore: {e}")

# --- Initialize SDK and Run Metadata Fetch ---
try:
    # Use environment variables set in the previous cell
    sdk = looker_sdk.init40()

    # We'll name our file "acs_census_metadata.json"
    METADATA_FILE = "acs_census_metadata.json"

    fetch_explore_metadata(sdk, MODEL_NAME, EXPLORE_NAME, METADATA_FILE)

except Exception as e:
    print(f"Failed to initialize Looker SDK or fetch metadata. Check credentials. Error: {e}")
    # In a real scenario, you'd stop here. For the notebook, we'll assume the file exists.

# Clean up the SDK connection
if 'sdk' in locals():
    # sdk.auth.logout() # This can sometimes hang, optional
    pass

Fetching metadata for data_block_acs_bigquery::acs_census_data...
Successfully fetched 360 fields.
Metadata saved to acs_census_metadata.json


### **2.2. Building the Looker Tool**

Now we build the Python function that will become our LangChain `StructuredTool`. This function (based on `tools/looker_tool.py`) is the core of the Looker connection.

It will:
1.  **Load** the metadata JSON we just created.
2.  **Define** a Pydantic schema (`LookerQueryInput`) to force the LLM to provide valid query parameters.
3.  **Define** the main query function (`_run_looker_query`) that authenticates, builds a `WriteQuery`, runs it, and formats an embed URL.
4.  **Wrap** this function in a `StructuredTool` for the agent to use.

In [ ]:
# --- 1. Helper function to load metadata ---
def _get_explore_metadata(metadata_file: str) -> str:
    """Loads the explore metadata JSON file and returns it as a string."""
    try:
        with open(metadata_file) as f:
            # We dump it into a string so it can be easily injected into the prompt
            return json.dumps(json.load(f))
    except FileNotFoundError:
        print(f"Fatal Error: {metadata_file} not found. Please run the metadata fetch cell.")
        return ""
    except Exception as e:
        print(f"Error loading {metadata_file}: {e}")
        return ""

# Load the metadata into a global variable for the tool prompt
EXPLORE_METADATA = _get_explore_metadata(METADATA_FILE)
if not EXPLORE_METADATA:
    print("Warning: Metadata is empty. The Looker agent will not know what fields to query.")

# --- 2. Pydantic Schema for LLM Input ---
class LookerQueryInput(BaseModel):
    """Input schema for running a Looker query."""
    fields: List[str] = Field(description="List of dimensions and measures, e.g., ['state.state_name', 'blockgroup.total_pop']")
    filters: Optional[Dict[str, str]] = Field(description="Dictionary of filters, e.g., {'state.state_name': 'California'}", default={})
    sorts: Optional[List[str]] = Field(description="List of fields to sort by, e.g., ['blockgroup.total_pop desc']", default=[])
    limit: Optional[str] = Field(description="Row limit for the query", default="500")
    vis_config_string: str = Field(
        description="A valid Looker vis_config JSON object, as a string. Use a simple 'table' viz if unsure.",
        default='{"type": "table"}'
    )

# --- 3. Core Looker Query Function ---
def _run_looker_query(
    fields: List[str],
    filters: Optional[Dict[str, str]] = {},
    sorts: Optional[List[str]] = [],
    limit: Optional[str] = "500",
    vis_config_string: str = '{"type": "table"}'
) -> str:
    """
    Runs a dynamic query to get data AND builds a full Expanded URL for embedding.
    This is the primary tool for all US Census data questions.

    (Based on tools/looker_tool.py)
    """
    if not fields:
        return json.dumps({"error": "You must provide at least one field."})

    sdk = None
    try:
        # 1. Initialize the SDK (it reads from os.environ)
        sdk = looker_sdk.init40()
    except Exception as e:
        return json.dumps({"error": f"Looker SDK not initialized. Check credentials. {e}"})

    try:
        # 2. Create the query payload
        vis_config_dict = json.loads(vis_config_string) # Parse string to dict

        query_payload = models40.WriteQuery(
            model=MODEL_NAME,
            view=EXPLORE_NAME,
            fields=fields,
            filters=filters,
            sorts=sorts,
            limit=str(limit),
            vis_config=vis_config_dict
        )

        # 3. Run the query inline to get the data
        print(f"--- Running Looker Inline Query --- \n{query_payload}")
        data_result = sdk.run_inline_query(
            result_format="json",
            body=query_payload
        )
        print("--- Data Query Successful ---")

        # 4. Build the Expanded URL parameters
        url_params = {}
        url_params['fields'] = ",".join(fields)
        url_params['sorts'] = ",".join(sorts)
        url_params['limit'] = str(limit)
        url_params['vis_config'] = vis_config_string
        url_params['toggle'] = 'vis' # Use the 'vis' tab

        filter_params = {}
        for key, value in filters.items():
            filter_params[f"f[{key}]"] = value

        url_params.update(filter_params)
        query_string = urlencode(url_params)

        # 5. Build the final Embed URL
        viz_url = f"{LOOKER_BROWSER_URL}/embed/explore/{MODEL_NAME}/{EXPLORE_NAME}?{query_string}"
        print(f"--- Expanded Embed URL Created: {viz_url} ---")

        # 6. Save the data to cache (for the Python agent)
        # We'll use the cache_tool functions from the original repo
        try:
            df = pd.read_json(io.StringIO(data_result))
            # Call the cache function directly
            cache_summary = save_data_to_cache.func(df, "data.csv")
            print(f"--- {cache_summary} ---")
            data_summary = f"Successfully queried and saved {len(df)} rows to data.csv."

            # --- NEW: Add data preview and stats for the agent ---
            # This is critical for the agent's "Insights" step
            data_preview = df.head().to_json(orient='records')
            data_stats = df.describe().to_json()

        except Exception as e:
            print(f"--- Failed to save to cache --- \n{e}")
            return json.dumps({"error": f"Error saving data to cache: {e}"})

        # 7. Return a summary, URL, preview, and stats back to the agent
        final_output = {
            "summary": data_summary,
            "viz_url": viz_url,
            "data_preview": data_preview,
            "data_stats": data_stats
        }
        return json.dumps(final_output)

    except Exception as e:
        print(f"--- Query Failed --- \n{e}")
        return json.dumps({"error": f"Error running Looker query: {e}"})
    finally:
        if sdk:
            # sdk.auth.logout() # Optional
            pass

### **2.3. Defining the Caching and Knowledge Tools**

The Looker agent needs two helper tools:
1.  `save_data_to_cache`: To save the query results.
2.  `get_census_data_definition`: To answer questions about what fields mean.

We also define `load_df_from_cache` for the Python agent later.

In [ ]:
# --- 1. Caching Tools ---
# (From tools/cache_tool.py)
# We need to define these so _run_looker_query can call them.

@tool
def save_data_to_cache(dataf: pd.DataFrame, file_path: str = "data.csv") -> str:
    """
    Saves a pandas DataFrame to a CSV file.
    This notebook uses 'data.csv' as the default.
    """
    try:
        dataf.to_csv(file_path, index=False)
        return f"Data saved to {file_path}"
    except Exception as e:
        return f"Error saving data: {e}"

@tool
def load_df_from_cache(file_path: str = "data.csv") -> pd.DataFrame:
    """
    Loads data from a CSV file into a pandas DataFrame.
    Returns None if the file is not found.
    """
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"Cache file not found: {file_path}")
        return None
    except Exception as e:
        print(f"Error loading cache: {e}")
        return None

# --- 2. Knowledge Tool ---
# (From tools/knowledge_tool.py)
# This is another tool the agent will use.

class TermInput(BaseModel):
    term: str = Field(description="The census term to define, e.g., 'poverty line' or 'median income'.")

@tool(args_schema=TermInput)
def get_census_data_definition(term: str) -> str:
    """
    Looks up the definition for a specific US Census term from the metadata file.
    Use this to answer questions like 'What is the poverty line?' or 'Define median income'.
    """
    try:
        with open(METADATA_FILE, 'r') as f:
            # The file from the repo is a list of dicts.
            metadata = json.load(f)

        search_term = term.lower().strip()

        for field in metadata:
            # Check 'name', 'label', and 'description' for the term
            if (search_term in field.get('name', '').lower() or
                search_term in field.get('label', '').lower() or
                search_term in field.get('description', '').lower()):
                return f"Definition for '{field.get('name')}' ({field.get('label')}): {field.get('description')}"

        return f"Sorry, I could not find a definition for the term '{term}'."

    except FileNotFoundError:
        return f"Error: The census metadata file ({METADATA_FILE}) was not found."
    except Exception as e:
        return f"Error reading census metadata: {e}"

# --- 3. Define the Final Looker Tool Wrapper ---
# This tool's description is CRITICAL. It includes the entire metadata schema,
# "grounding" the LLM on what fields are available.
# (This logic is from app.py's `create_looker_agent_chain`)

looker_data_tool = StructuredTool.from_function(
    func=_run_looker_query, # Use our main function
    name="LookerDataQuery",
    description=(
        "Use this tool as the **primary source** for ANY questions about US population or census demographics. "
        "This includes nationwide totals (e.g., 'What is the total population of the US?') as well as specific breakdowns "
        "by geography (e.g., 'What is the population in California?', 'List median income by county in Texas').\n\n"

        "**CRITICAL QUERY-BUILDING INSTRUCTIONS:**\n"
        "1.  To get a total, aggregated value (like 'total population'), you **MUST** use the 'measure' field (e.g., `blockgroup.total_pop`).\n"
        "2.  For a general nationwide 'total population', you MUST use the measure `blockgroup.total_pop` and apply no filters.\n"
        "3.  To group data by a category (like 'by state'), you must add that 'dimension' (e.g., `state.state_name`) to the `fields` list along with the measure.\n"
        "4.  You MUST also generate a `vis_config_string`. This is a JSON object *as a string*.\n"
        "   - For single numbers (like total population), use: `'{\"type\": \"single_value\"}'`\n"
        "   - For tables (data grouped by state), use: `'{\"type\": \"table\"}'`\n"
        "   - For maps (data by state), use: `'{\"type\": \"looker_map\", \"map_field_name\": \"state.state_name\"}'`\n"
        "   - For bar charts (pop by state), use: `'{\"type\": \"looker_bar\", \"stacking\": \"normal\"}'`\n"
        "   - If unsure, default to: `'{\"type\": \"table\"}'`\n\n"

        f"Here is the complete schema of available fields: {EXPLORE_METADATA}"
    ),
    args_schema=LookerQueryInput
)

---

## **Section 3: The LLM Brain - A Simple Looker Agent**

Now that we have our `looker_data_tool` and `get_census_data_definition` tool, we can build our first agent. This simple agent will *only* have access to these two tools. This is a great way to test the Looker connection in isolation.

This section is based on `02_test_looker_agent.py`.

In [ ]:
# --- 1. Initialize the LLMs ---
# We use two models:
# - Pro: For heavy reasoning (Python, General Search)
# - Flash: For fast, structured tasks (Looker, Routing)
# (From app.py)

try:
    llm_pro = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro", # Use a powerful model for reasoning
        google_api_key=os.environ["GOOGLE_API_KEY"],
        temperature=0,
        convert_system_message_to_human=True
    )

    llm_flash = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash", # Use a fast model for structured output
        google_api_key=os.environ["GOOGLE_API_KEY"],
        temperature=0,
        convert_system_message_to_human=True
    )

    print("LLMs initialized (Pro and Flash).")
except Exception as e:
    print(f"Error initializing LLMs. Check GOOGLE_API_KEY. Error: {e}")
    # This is a critical step, so we'll raise an error to stop the notebook
    raise e

LLMs initialized (Pro and Flash).


### **3.1. Build the Simple Looker Agent**

This is the prompt that tells the agent *how* to behave. Note the "ANALYST STRATEGY" and "FINAL ANSWER" sections, which are custom instructions to improve the agent's output.

In [ ]:
LOOKER_AGENT_PROMPT_TEMPLATE = """
You are an expert assistant for US Census data. You have two tools:
1.  `LookerDataQuery`: To get data and visualizations.
2.  `get_census_data_definition`: To define terms.

**ANALYST STRATEGY:**
-   You are a senior data analyst. Your goal is to answer the user's question with the *most relevant* data.
-   **If the user asks a strategic question (e.g., "where should I open a bookstore?"), DO NOT just query `blockgroup.total_pop`.**
-   Instead, THINK about what data implies a good market. For a bookstore, good fields would be `blockgroup.bachelors_degree`, `blockgroup.masters_degree`, or `blockgroup.median_income_dim`.
-   Select fields that *actually* answer the user's strategic question.

**STRATEGY FOR `LookerDataQuery`:**
-   You MUST provide a `vis_config_string` (a JSON string).
-   **CHART PREFERENCE:** Prefer `'{{\"type\": \"looker_column\"}}'` or `'{{\"type\": \"looker_bar\"}}'` for most queries that compare values.
-   Use `'{{\"type\": \"single_value\"}}'` for single-number answers.

**STRATEGY FOR FINAL ANSWER:**
-   Your tool will return a JSON object with: "summary", "viz_url", "data_preview", and "data_stats".
-   Your "Final Answer" to the user must be a 1-2 paragraph response, formatted in MARKDOWN with two headings: '### Summary' and '### Insights'.
-   Under '### Summary', describe the data that was retrieved.
-   Under '### Insights', use the "data_stats" (for min/max/mean) and "data_preview" (for string examples) to provide a brief, *specific* analysis.
-   **IMPORTANT**: Include the "viz_url" in your final answer so the user can see it.

Here are the tools you must use:
{tools}

Here are the names of your tools: {tool_names}

Use the following format for your thoughts and actions.
(You MUST use this format. Do not just output the final answer.)

Thought:
(Your reasoning and plan)
Action:
```json
{{
  "action": "Your chosen tool",
  "action_input": {{"parameter": "value"}}
}}
```
Observation:
(The tool's JSON output)
Thought:
(Your analysis of the tool's output)
Action:
```json
{{
  "action": "Final Answer",
  "action_input": "### Summary\n\n(Your summary here)\n\n### Insights\n\n(Your insights here)\n\nHere is the link to the visualization: [Looker Viz](viz_url_here)"
}}
```

CHAT HISTORY:
{chat_history}

USER INPUT: {input}

Begin!
Thought:
{agent_scratchpad}
"""


### **3.2. Test the Simple Looker Agent**

Let's run the agent we just created.

**Note:** This will make a **LIVE API CALL** to Looker using the credentials you provided in Section 1.

In [ ]:
def create_simple_looker_agent():
    """Creates an AgentExecutor that can only use the Looker and Knowledge tools."""

    tools = [looker_data_tool, get_census_data_definition]

    agent_prompt = PromptTemplate(
        template=LOOKER_AGENT_PROMPT_TEMPLATE,
        input_variables=["input", "chat_history", "agent_scratchpad"],
        partial_variables={
            "tools": render_text_description(tools),
            "tool_names": ", ".join([t.name for t in tools])
        }
    )

    # We use the fast 'flash' model for the Looker agent
    agent = create_structured_chat_agent(llm_flash, tools, agent_prompt)

    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True, # Set to True to see the agent's "thoughts"
        handle_parsing_errors=True,
        max_iterations=8
    ).with_config({"run_name": "LookerAgent"})

    return agent_executor

In [ ]:
simple_looker_agent = create_simple_looker_agent()

# --- 2. Test 1: A definition question ---
print("--- TEST 1: Definition Query ---")
response_1 = simple_looker_agent.invoke({
    "input": "What does 'blockgroup.total_pop' mean?",
    "chat_history": []
})
print(f"--- FINAL AGENT OUTPUT 1 ---\n{response_1['output']}")

# %%
# --- 3. Test 2: A data question ---
print("\n\n--- TEST 2: Data Query ---")
response_2 = simple_looker_agent.invoke({
    "input": "What is the total population of the US?",
    "chat_history": []
})
print(f"--- FINAL AGENT OUTPUT 2 ---\n{response_2['output']}")

--- TEST 1: Definition Query ---


> Entering new LookerAgent chain...
The user is asking for the definition of a specific census term, 'blockgroup.total_pop'. I should use the `get_census_data_definition` tool to retrieve this information.
Action:
```json
{
  "action": "get_census_data_definition",
  "action_input": {
    "term": "blockgroup.total_pop"
  }
}
```Definition for 'blockgroup.total_pop_dim' (Total Pop Dim): Total Population. The total number of all people living in a given geographic area.  This is a very useful catch-all denominator when calculating rates.The tool returned the definition for 'blockgroup.total_pop_dim', which is "Total Population. The total number of all people living in a given geographic area. This is a very useful catch-all denominator when calculating rates." This directly answers the user's question. I will now format this into the final answer.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "### Summary\n\nThe term 'blockgroup.total_

In [ ]:
# --- 3. Test 2: A data question ---
print("\n\n--- TEST 2: Data Query ---")
response_2 = simple_looker_agent.invoke({
    "input": "What is the total population of the US?",
    "chat_history": []
})
print(f"--- FINAL AGENT OUTPUT 2 ---\n{response_2['output']}")



--- TEST 2: Data Query ---


> Entering new LookerAgent chain...
The user is asking for the total population of the US. I need to use the `LookerDataQuery` tool to get this information. The relevant field for total population is `blockgroup.total_pop`. Since the user is asking for the total US population, I will not apply any filters and will use a `single_value` visualization.
Action:
```json
{
  "action": "LookerDataQuery",
  "action_input": {
    "fields": ["blockgroup.total_pop"],
    "vis_config_string": "{\"type\": \"single_value\"}"
  }
}
```--- Running Looker Inline Query --- 
WriteQuery(model='data_block_acs_bigquery', view='acs_census_data', fields=['blockgroup.total_pop'], pivots=None, fill_fields=None, filters={}, filter_expression=None, sorts=[], limit='500', column_limit=None, total=None, row_total=None, subtotals=None, vis_config={'type': 'single_value'}, filter_config=None, visible_ui_sections=None, dynamic_fields=None, client_id=None, query_timezone=None)
--- Data Qu

---

## **Section 4: Building the Multi-Agent System**

The simple agent is great, but it has a problem. What if the user asks an analysis question, like "What's the *average* population?" or "Compare the population of California and Texas." The Looker agent isn't trained for that.

To solve this, we create a **multi-agent system** (from `app.py`):
1.  **Router:** A fast LLM (`llm_flash`) that classifies the user's query and routes it to the correct agent.
2.  **Looker Agent:** The agent we just built, which is responsible for *data retrieval*.
3.  **Python Agent:** A *new* agent (`llm_pro`) that is responsible for *data analysis*. Its only tool is a Python REPL that can read the `data.csv` file saved by the Looker agent.
4.  **General Agent:** A fallback agent for small talk and web searches.

### **4.1. The Router**

This chain doesn't answer the question; it just classifies it.

In [ ]:
# --- 1. Pydantic Model for Routing ---
class RouteQuery(BaseModel):
    """Classifies the user's query to the appropriate agent."""
    destination: Literal["looker", "python_agent", "general"] = Field(
        ...,
        description=(
            "The agent to route the query to. "
            "'looker' for data retrieval/visualization about US census data (e.g., 'what is the population...'). "
            "'python_agent' for analysis, math, or manipulation on *already cached* data (e.g., 'what is the average...', 'compare X and Y'). "
            "'general' for conversation, definitions, web searches, or anything else."
        )
    )

# --- 2. Router Prompt ---
router_prompt_template = """
You are an expert dispatcher routing user queries to the correct agent.
Based on the user's query and chat history, you must classify it into one of the following destinations.
Your response must be a JSON object matching the 'RouteQuery' Pydantic schema.

{schema}

CHAT HISTORY:
{chat_history}

USER QUERY:
{input}

CLASSIFICATION:
"""

# --- 3. Router Chain ---
parser = PydanticOutputParser(pydantic_object=RouteQuery)

router_prompt = PromptTemplate(
    template=router_prompt_template,
    input_variables=["input", "chat_history"],
    partial_variables={"schema": parser.get_format_instructions()}
)

# Use the fast model for the router
router = router_prompt | llm_flash | parser

# --- 4. Test the Router ---
print("--- Testing Router ---")
query_1 = "What is the population by state?"
route_1 = router.invoke({"input": query_1, "chat_history": []})
print(f"Query: '{query_1}' -> Route: {route_1.destination}")

query_2 = "What is the average population from that data?"
# We simulate chat history to give context
history_2 = [
    HumanMessage(content=query_1),
    AIMessage(content="I have retrieved the population by state and saved it to data.csv.")
]
route_2 = router.invoke({"input": query_2, "chat_history": history_2})
print(f"Query: '{query_2}' -> Route: {route_2.destination}")

query_3 = "Hello, how are you?"
route_3 = router.invoke({"input": query_3, "chat_history": []})
print(f"Query: '{query_3}' -> Route: {route_3.destination}")

query_4 = "What's the weather today?"
route_4 = router.invoke({"input": query_4, "chat_history": []})
print(f"Query: '{query_4}' -> Route: {route_4.destination}")

--- Testing Router ---
Query: 'What is the population by state?' -> Route: looker
Query: 'What is the average population from that data?' -> Route: python_agent
Query: 'Hello, how are you?' -> Route: general
Query: 'What's the weather today?' -> Route: general


### **4.2. The Python Agent (Data Analyst)**

This agent is an expert in Python and pandas. Its prompt *commands* it to load `data.csv` first, then perform analysis.

In [34]:
PYTHON_AGENT_PROMPT_TEMPLATE = """
You are an expert Python data analyst. You have access to a Python REPL tool.
Your primary task is to answer the user's question by analyzing a pandas DataFrame named `df`.

**CRITICAL CONTEXT:**
- A file named `data.csv` exists in the local directory.
- Your **FIRST ACTION** must *always* be to load this file into a pandas DataFrame named `df`.
  Example: `import pandas as pd\ndf = pd.read_csv('data.csv')`
- After loading, your second action should *always* be to inspect the columns: `print(df.columns)`
- **DO NOT** assume `df` is already loaded. You must load it *every time*.
- The pandas library is available as `pd`.

**STRATEGY FOR FINAL ANSWER:**
- Your final answer should be conversational, explaining what you found.
- Do not just output the raw number. Explain what it means.

Here are the tools you must use:
{tools}

Here are the names of your tools: {tool_names}

Use the following format:

Thought:
(Your plan: load data, inspect, then analyze)
Action:
```json
{{
  "action": "Python_REPL",
  "action_input": "import pandas as pd\ndf = pd.read_csv('data.csv')\nprint(df.columns)"
}}
```
Observation:
(The tool's output, e.g., "Index(['state.state_name', ...])")
Thought:
(Your plan to analyze the correct columns)
Action:
```json
{{
  "action": "Python_REPL",
  "action_input": "result = df['blockgroup.total_pop'].mean()\nprint(result)"
}}
```
Observation:
(The tool's output, e.g., "12345.67")
Thought:
(Your plan to formulate a final, conversational answer)
Action:
```json
{{
  "action": "Final Answer",
  "action_input": "I've finished the analysis. The average population from the data is 12,345.67."
}}
```

CHAT HISTORY:
{chat_history}

USER INPUT: {input}

Begin!
Thought:
{agent_scratchpad}
"""


### **4.3. The General Agent (Fallback)**

This agent handles everything else, including general knowledge and web searches.

In [35]:
# %%
# --- 1. General Agent Prompt ---
GENERAL_AGENT_PROMPT_TEMPLATE = """
You are a helpful assistant.
Your goal is to provide helpful, conversational answers to the user's question.
You have tools for looking up specific US Census definitions and for general web searches.

Here are the available tools:
{tools}

Use the following format:

Thought: (Your reasoning and plan)
Action:
```json
{{
  "action": "Your chosen tool",
  "action_input": {{"parameter": "value"}}
}}
```
Observation: (The tool's output)
Thought: I have the information. I will now answer the user.
Action:
```json
{{
  "action": "Final Answer",
  "action_input": "The answer to the user's question."
}}
```

USER INPUT: {input}
CHAT HISTORY:
{chat_history}

Begin!
Thought:
{agent_scratchpad}
"""

### **4.4. The Main Graph (Tying It All Together)**

This `RunnableBranch` is the final piece. It takes the *output* of the `router` and selects which agent `AgentExecutor` to run.

In [40]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [45]:
# --- 1. Create all the agent chains ---
looker_chain = create_simple_looker_agent()
python_chain = create_python_agent_chain()
general_chain = create_general_agent_chain()

# --- 2. Define the Branch ---
# This is the "Supervisor" from ARCHITECTURE.md
branch = RunnableBranch(
    (lambda x: x['route'].destination == "looker", looker_chain),
    (lambda x: x['route'].destination == "python_agent", python_chain),
    (lambda x: x['route'].destination == "general", general_chain),
    general_chain # Default fallback
)

# --- 3. Define the Full Chain ---
# The full chain pipes the input to the router,
# then passes the input + route to the branch.

full_chain = (
    RunnablePassthrough.assign(
        # Step 1: Call the router. The result is added to the
        # input dict under the key 'route'.
        route=lambda x: router.invoke(x)
    )
    | branch # Step 2: Call the selected agent
)

---

## **Section 5: Running the Full System**

Let's simulate the two-step conversation from our router test.

1.  **User:** "Show me population by state."
    *   **Router** -> `looker`
    *   **Looker Agent** -> Runs query, saves `data.csv`, returns summary and URL.
2.  **User:** "What is the average population from that data?"
    *   **Router** (with history) -> `python_agent`
    *   **Python Agent** -> Loads `data.csv`, calculates `.mean()`, returns analysis.

In [42]:
chat_history = []

# --- Conversation Turn 1: Looker Query ---
print("="*40 + "\nTURN 1: LOOKER QUERY\n" + "="*40)
query_1 = "Show me total population and median income by state, sorted by population desc"

# Run the full chain
response_1 = full_chain.invoke({
    "input": query_1,
    "chat_history": chat_history
})

# Update our history
chat_history.append(HumanMessage(content=query_1))
chat_history.append(AIMessage(content=response_1['output']))

print(f"\n--- FINAL AGENT 1 OUTPUT ---\n{response_1['output']}")

TURN 1: LOOKER QUERY


> Entering new LookerAgent chain...
The user is asking for total population and median income by state, sorted by population in descending order. I need to use the `LookerDataQuery` tool to retrieve this information.

Here's my plan:
1.  **Fields**: I need `state.state_name` for the state, `blockgroup.total_pop` for total population, and `blockgroup.median_income_dim` for median income.
2.  **Filters**: No specific filters are mentioned, so I will not apply any.
3.  **Sorts**: The user requested sorting by population in descending order, so I'll use `blockgroup.total_pop desc`.
4.  **Limit**: The default limit of 500 should be sufficient for all states.
5.  **Visualization**: A bar chart or table would be suitable for comparing states. I'll choose a bar chart to easily visualize the population differences, and include median income as a secondary measure. I will use `state.state_name` as the dimension and `blockgroup.total_pop` and `blockgroup.median_income_dim` 

In [43]:
# --- Conversation Turn 2: Python Agent Query ---
print("="*40 + "\nTURN 2: PYTHON ANALYSIS\n" + "="*40)
query_2 = "What is the average median income and max population from that data?"

# Run the full chain
response_2 = full_chain.invoke({
    "input": query_2,
    "chat_history": chat_history
})

# Update our history
chat_history.append(HumanMessage(content=query_2))
chat_history.append(AIMessage(content=response_2['output']))

print(f"\n--- FINAL AGENT 2 OUTPUT ---\n{response_2['output']}")

TURN 2: PYTHON ANALYSIS


> Entering new PythonAgent chain...


Thought:
The user wants to know the average median income and the maximum population from the dataset.

My plan is:
1.  Load the `data.csv` file into a pandas DataFrame named `df`.
2.  Print the column names to identify the correct columns for population and median income.
3.  Calculate the mean of the median income column.
4.  Calculate the maximum value of the population column.
5.  Combine these two results into a final, conversational answer.
Action:
```json
{
  "action": "Python_REPL",
  "action_input": "import pandas as pd\ndf = pd.read_csv('data.csv')\nprint(df.columns)"
}
```Index(['state.state_name', 'state.state_median_income_dim',
       'blockgroup.total_pop'],
      dtype='object')
The user wants to find the average median income and the maximum population.
From the previous step, I have the column names:
- `'state.state_median_income_dim'` for median income.
- `'blockgroup.total_pop'` for population.

Now I will calculate the average of `'state.state_median_income_dim'` a

In [44]:
# --- Conversation Turn 3: General Query ---
print("="*40 + "\nTURN 3: GENERAL KNOWLEDGE\n" + "="*40)
query_3 = "What is the weather in New York?"

# Run the full chain
response_3 = full_chain.invoke({
    "input": query_3,
    "chat_history": chat_history
})

# Update our history
chat_history.append(HumanMessage(content=query_3))
chat_history.append(AIMessage(content=response_3['output']))

print(f"\n--- FINAL AGENT 3 OUTPUT ---\n{response_3['output']}")

TURN 3: GENERAL KNOWLEDGE


> Entering new GeneralAgent chain...
Thought: The user is asking for the weather in New York. I do not have a specific weather tool, but I can use the `DuckDuckGoSearchRun` tool to get this information from the web. I will search for "weather in New York" to get the current conditions.
Action:
```json
{
  "action": "DuckDuckGoSearchRun",
  "action_input": {
    "query": "weather in New York"
  }
}
```2 weeks ago - In the Holdridge life zones system, having a mean annual biotemperature above 53.6 °F (12 °C) and mean annual precipitation above 39.4 inches (1,000 mm), New York City features a warm temperate moist forest climate . By the Trewartha classification, the city is defined as having a Temperate ... New York - Weather warnings issued. 14-day forecast.Observations - New York Downtown Manhattan Heli. Observed at 14:00, Saturday 6 September. Current weather in New York and forecast for today, tomorrow, and next 14 days. Use Current Location. Recent. New Yo

## **Conclusion**

This notebook has demonstrated the core, frontend-agnostic logic of the Looker-Gemini chatbot. We successfully:
1.  **Fetched Metadata** from a Looker Explore to ground the LLM.
2.  **Built a `looker_data_tool`** that queries Looker, generates an embed URL, and caches the data to `data.csv`.
3.  **Built a `PythonREPLTool`** that can securely analyze the cached `data.csv`.
4.  **Created a `Router`** to classify user intent.
5.  **Assembled a Multi-Agent System** using a `RunnableBranch` that directs traffic to a specialized **Looker Agent**, **Python Agent**, or **General Agent**, creating a seamless conversational data experience.